In [1]:
import os
import sys
import random
import time
import argparse
import numpy as np
import matplotlib.pyplot as plt

import multiprocessing as mp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from models import DPINet
from data import *
#from data import PhysicsFleXDataset, collate_fn

from utils import count_parameters

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--pstep', type=int, default=2)
parser.add_argument('--n_rollout', type=int, default=0)
parser.add_argument('--time_step', type=int, default=0)
parser.add_argument('--time_step_clip', type=int, default=0)
parser.add_argument('--dt', type=float, default=1./60.)
parser.add_argument('--nf_relation', type=int, default=300)
parser.add_argument('--nf_particle', type=int, default=200)
parser.add_argument('--nf_effect', type=int, default=200)
parser.add_argument('--env', default='')
parser.add_argument('--train_valid_ratio', type=float, default=0.9)
parser.add_argument('--outf', default='files')
parser.add_argument('--dataf', default='data')
parser.add_argument('--num_workers', type=int, default=10)
parser.add_argument('--gen_data', type=int, default=0)
parser.add_argument('--gen_stat', type=int, default=0)
parser.add_argument('--log_per_iter', type=int, default=1000)
parser.add_argument('--ckp_per_iter', type=int, default=10000)
parser.add_argument('--eval', type=int, default=0)
parser.add_argument('--verbose_data', type=int, default=1)
parser.add_argument('--verbose_model', type=int, default=1)

parser.add_argument('--n_instance', type=int, default=0)
parser.add_argument('--n_stages', type=int, default=0)
parser.add_argument('--n_his', type=int, default=0)

parser.add_argument('--n_epoch', type=int, default=1000)
parser.add_argument('--beta1', type=float, default=0.9)
parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--batch_size', type=int, default=1)
parser.add_argument('--forward_times', type=int, default=2)

parser.add_argument('--resume_epoch', type=int, default=0)
parser.add_argument('--resume_iter', type=int, default=0)

# shape state:
# [x, y, z, x_last, y_last, z_last, quat(4), quat_last(4)]
parser.add_argument('--shape_state_dim', type=int, default=14)

# object attributes:
parser.add_argument('--attr_dim', type=int, default=0)

# object state:
parser.add_argument('--state_dim', type=int, default=0)
parser.add_argument('--position_dim', type=int, default=0)

# relation attr:
parser.add_argument('--relation_dim', type=int, default=0)

_StoreAction(option_strings=['--relation_dim'], dest='relation_dim', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help=None, metavar=None)

In [3]:
args = parser.parse_args("--env SingleHair --gen_data 0".split())

In [4]:
phases_dict = dict()

args.n_rollout = 50
args.num_workers = 3
args.gen_stat = 1

args.dataf = 'data'

# object states:
# [x, y, z, xdot, ydot, zdot]
args.state_dim = 6
args.position_dim = 3

# object attr:
# [rigid]
args.attr_dim = 1

# relation attr:
# [none]
args.relation_dim = 1

args.time_step = 500
args.time_step_clip = 20
args.n_instance = 1
args.n_stages = 1

args.neighbor_radius = 0.08

phases_dict["instance_idx"] = [0, 30]
phases_dict["root_num"] = [[]]
phases_dict["instance"] = ['solid']
phases_dict["material"] = ['solid']

data_names = ['positions', 'velocities','hair_idx']
verbose = 0
phase = 'train'
data_dir = os.path.join(args.dataf, phase)
stat_path = os.path.join(args.dataf, 'stat.h5')
n_rollout = 10



info = {
    'env': args.env,
    'root_num': phases_dict['root_num'],
    'thread_idx': 0,
    'data_dir': data_dir,
    'data_names': data_names,
    'n_rollout': n_rollout // args.num_workers,
    'n_instance': args.n_instance,
    'time_step': args.time_step,
    'time_step_clip': args.time_step_clip,
    'dt': args.dt,
    'shape_state_dim': args.shape_state_dim}

info['env_idx'] = 11

args.outf = 'dump_SingleHair/' + args.outf

args.outf = args.outf + '_' + args.env
args.dataf = 'data/' + args.dataf + '_' + args.env
print (args.dataf)
os.system('mkdir -p ' + args.outf)
os.system('mkdir -p ' + args.dataf)

data/data_SingleHair


0

In [5]:
from data import *

def gen_PyFleX(info):

    env, root_num = info['env'], info['root_num']
    thread_idx, data_dir, data_names = info['thread_idx'], info['data_dir'], info['data_names']
    n_rollout, n_instance = info['n_rollout'], info['n_instance']
    time_step, time_step_clip = info['time_step'], info['time_step_clip']
    shape_state_dim, dt = info['shape_state_dim'], info['dt']

    env_idx = info['env_idx'] # =11

    np.random.seed(round(time.time() * 1000 + thread_idx) % 2**32)
    
    stats = [init_stat(3), init_stat(3)]

    import pyflex
    pyflex.init()

    for i in range(n_rollout):

        if i % 10 == 0:
            print("%d / %d" % (i, n_rollout))

        rollout_idx = thread_idx * n_rollout + i
        rollout_dir = os.path.join(data_dir, str(rollout_idx))
        os.system('mkdir -p ' + rollout_dir)
        
        # scene_params: [len(box) at dim x,len(box) at dim y,len(box) at dim z, num_hair per circle, num_circle]
        cap_size = [0.1,1.5]
        N_hairs = 1


        scene_params = np.array(cap_size)

        pyflex.set_scene(env_idx, scene_params, thread_idx)
        n_particles = pyflex.get_n_particles()
        n_shapes = 1
        N_particles_per_hair = int(n_particles/N_hairs)
        idx_begins = np.arange(N_hairs)*N_particles_per_hair
        idx_hairs = [[i,i+N_particles_per_hair-1] for i in idx_begins]

        positions = np.zeros((time_step, n_particles+ n_shapes, 3), dtype=np.float32)
        velocities = np.zeros((time_step, n_particles+ n_shapes, 3), dtype=np.float32)
   #     shape_position = np.zeros((time_step, n_shapes, 3), dtype=np.float32)
    #    shape_velocities = np.zeros((time_step, n_shapes, 3), dtype=np.float32)

        for j in range(time_step_clip):
         #   p_clip = pyflex.get_positions().reshape(-1, 4)[:, :3]
         #   shape_p_clip = pyflex.get_shape_states()[:3].reshape(-1,3)
            p_clip = np.concatenate([pyflex.get_positions().reshape(-1, 4)[:, :3],pyflex.get_shape_states()[:3].reshape(-1,3)],axis = 0)
            pyflex.step()

        for j in range(time_step):
            positions[j, :n_particles] = pyflex.get_positions().reshape(-1, 4)[:, :3]
            for k in range(n_shapes):
                 positions[j, n_particles + k] = pyflex.get_shape_states()[:3]
        #    shape_position[j] = pyflex.get_shape_states()[:3].reshape(-1,3)
        #    shape_prevposition = pyflex.get_shape_states()[3:6].reshape(-1,3)
            if j == 0:
                velocities[j] = (positions[j] - p_clip) / dt
           #     shape_velocities[j] = (shape_position[j] - shape_p_clip)/dt
            else:
                velocities[j] = (positions[j] - positions[j - 1]) / dt
          #      shape_velocities[j] = (shape_position[j] - shape_position[j-1])/dt

            pyflex.step()
            data = [positions[j], velocities[j], idx_hairs]
            store_data(data_names, data, os.path.join(rollout_dir, str(j) + '.h5'))
        
        # change dtype for more accurate stat calculation
        # only normalize positions and velocities
        datas = [positions.astype(np.float64), velocities.astype(np.float64)]

        for j in range(len(stats)): 
            # here j = 2, refers to positions and velocities
            stat = init_stat(stats[j].shape[0]) 
            # stat= np.zeros((3,3))
            stat[:, 0] = np.mean(datas[j], axis=(0, 1))[:]
            stat[:, 1] = np.std(datas[j], axis=(0, 1))[:]
            stat[:, 2] = datas[j].shape[0] * datas[j].shape[1] # time_step*n_particles
            stats[j] = combine_stat(stats[j], stat)

    pyflex.clean()

    return stats

In [6]:
class PhysicsFleXDataset(Dataset):

    def __init__(self, args, phase, phases_dict, verbose):
        self.args = args
        self.phase = phase
        self.phases_dict = phases_dict
        self.verbose = verbose
        self.data_dir = os.path.join(self.args.dataf, phase)
        self.stat_path = os.path.join(self.args.dataf, 'stat.h5')

        os.system('mkdir -p ' + self.data_dir)

        #    self.data_names = ['positions', 'velocities', 'shape_quats', 'clusters', 'scene_params']
        self.data_names = ['positions', 'velocities','hair_idx']

        ratio = self.args.train_valid_ratio
        if phase == 'train':
            self.n_rollout = int(self.args.n_rollout * ratio)
        elif phase == 'valid':
            self.n_rollout = self.args.n_rollout - int(self.args.n_rollout * ratio)
        else:
            raise AssertionError("Unknown phase")

    def __len__(self):
        return self.n_rollout * (self.args.time_step - 1)

    def load_data(self, name):
        self.stat = load_data(self.data_names[:2], self.stat_path)
        for i in range(len(self.stat)):
            self.stat[i] = self.stat[i][-self.args.position_dim:, :]
            # print(self.data_names[i], self.stat[i].shape)

    def gen_data(self, name):
        # if the data hasn't been generated, generate the data
        print("Generating data ... n_rollout=%d, time_step=%d" % (self.n_rollout, self.args.time_step))

        infos = []
        for i in range(self.args.num_workers):
            info = {
                'env': self.args.env,
                'root_num': self.phases_dict['root_num'],
                'thread_idx': i,
                'data_dir': self.data_dir,
                'data_names': self.data_names,
                'n_rollout': self.n_rollout // self.args.num_workers,
                'n_instance': self.args.n_instance,
                'time_step': self.args.time_step,
                'time_step_clip': self.args.time_step_clip,
                'dt': self.args.dt,
                'shape_state_dim': self.args.shape_state_dim}

            info['env_idx'] = 11
            infos.append(info)

        cores = self.args.num_workers
        pool = mp.Pool(processes=cores)
        data = pool.map(gen_PyFleX, infos)

        print("Training data generated, warpping up stats ...")

        if self.phase == 'train' and self.args.gen_stat:
            # positions [x, y, z], velocities[xdot, ydot, zdot]
            self.stat = [init_stat(3), init_stat(3)]
            for i in range(len(data)):
                for j in range(len(self.stat)):
                    self.stat[j] = combine_stat(self.stat[j], data[i][j])
            store_data(self.data_names[:2], self.stat, self.stat_path)
        else:
            print("Loading stat from %s ..." % self.stat_path)
            self.stat = load_data(self.data_names[:2], self.stat_path)

    def __getitem__(self, idx):
        idx_rollout = idx // (self.args.time_step - 1)
        idx_timestep = idx % (self.args.time_step - 1)

        # ignore the first frame for env RiceGrip
        if self.args.env == 'RiceGrip' and idx_timestep == 0:
            idx_timestep = np.random.randint(1, self.args.time_step - 1)

        data_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep) + '.h5')
        data_nxt_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep + 1) + '.h5')

        data = load_data(self.data_names, data_path)

        '''
        vel_his = []
        for i in range(self.args.n_his):
            path = os.path.join(self.data_dir, str(idx_rollout), str(max(1, idx_timestep - i - 1)) + '.h5')
            data_his = load_data(self.data_names, path)
            vel_his.append(data_his[1])

        data[1] = np.concatenate([data[1]] + vel_his, 1)
        '''
        
        attr, state, relations, n_particles, n_shapes = \
                prepare_input(data, self.stat, self.args, self.phases_dict, self.verbose)

        ### label
        data_nxt = normalize(load_data(self.data_names, data_nxt_path), self.stat)

        label = torch.FloatTensor(data_nxt[1][:n_particles])

        return attr, state, relations, n_particles, n_shapes, label

In [7]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):
    '''
    for a single hair
    '''
    positions, velocities, hairs_idx = data
    n_shapes = 1
    hairs_idx_begin = [idx[0] for idx in hairs_idx]
    n_particles = positions.shape[0] - n_shapes
    R = 0.1
    
    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    #   here we only consider the hairs but not the gripper, attr_dim = 1, attr = 0 for hair, attr = 1 for shapes
    attr = np.zeros((n_particles+n_shapes, args.attr_dim))
    
    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps
    
    ##### add env specific graph components
    ### specify for shapes
    rels = []
    vals = []
    
    for i in range(n_shapes):
        attr[n_particles+i, 0] = 1
        dis = np.linalg.norm(positions[:n_particles,:2]-positions[n_particles+i,:2],axis = 1)
        nodes_rel = np.nonzero(dis <= R)[0]
        # for relation between hair nodes and a gripper, we note it as 1
        gripper = np.ones(nodes_rel.shape[0], dtype=np.int) * (n_particles+i)
        rels += [np.stack([nodes_rel, gripper, np.ones(nodes_rel.shape[0])], axis=1)]
        vals += [np.ones(nodes_rel.shape[0], dtype=np.int)]
        
    
    ##### add relations between leaf particles
    ## here we only consider the relations in a hair: the relation between a node and the nodes nearby
    ## simple case for one hair, TEMPORARY 2 rels for one link
    nodes_p = np.arange(n_particles-1)
    val = np.linalg.norm(positions[1:n_particles]-positions[:n_particles-1],axis = 1)
    R1 = np.stack([nodes_p,nodes_p+1, np.zeros(n_particles-1)],axis = 1)
    R2 = np.stack([nodes_p+1,nodes_p, np.zeros(n_particles-1)],axis = 1)
    rels += [np.concatenate([R1,R2],axis = 0)]
    vals += [val,val]
    
    rels = np.concatenate(rels, 0)
    vals = np.concatenate(vals, 0)
    
  #  print (vals.shape)
 #   print (rels.shape)
    
    
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        # Ra: relation attributes
    #    Ra = np.zeros((rels.shape[0], args.relation_dim))  
        Ra = rels[:,2].reshape([-1,1])
        Ras.append(torch.FloatTensor(Ra))
        # values could be changed
     #   values.append(torch.FloatTensor([1] * rels.shape[0]))
      #  values.append(rels[:,2])
        #### for hairs: values equals to the length of this segment
        values.append(torch.FloatTensor(vals))
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)
        
        
    if verbose:
        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))
        
        
    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))
        
    state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))
    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes#, instance_idx



In [7]:
datasets = {phase: PhysicsFleXDataset(
    args, phase, phases_dict, args.verbose_data) for phase in ['train', 'valid']}
datasets['train'].load_data(args.env)

In [13]:
dataloaders = {x: torch.utils.data.DataLoader(
    datasets[x], batch_size=args.batch_size,
    shuffle=True if x == 'train' else False,
    num_workers=args.num_workers,
    collate_fn=collate_fn)
    for x in ['train', 'valid']}

In [21]:
from models import *

class DPINet(nn.Module):
    def __init__(self, args, stat, phases_dict, residual=False, use_gpu=False):
        super(DPINet, self).__init__()

        self.args = args

        state_dim = args.state_dim
        attr_dim = args.attr_dim
        relation_dim = args.relation_dim
        nf_particle = 100#args.nf_particle
        nf_relation = 150 #args.nf_relation
        nf_effect = 150 #args.nf_effect

        self.nf_effect = nf_effect

        self.stat = stat
        self.use_gpu = use_gpu
        self.residual = residual 

        # (1) particle attr (2) state
        self.particle_encoder = ParticleEncoder(attr_dim + state_dim, nf_particle, nf_effect)

        # (1) sender attr (2) receiver attr (3) state receiver (4) state_diff (5) relation attr
        self.relation_encoder = RelationEncoder( 2 * attr_dim + 2 * state_dim + relation_dim, nf_relation, nf_relation)

        # (1) relation encode (2) sender effect (3) receiver effect
        self.relation_propagator = Propagator(nf_relation + 2 * nf_effect, nf_effect)

        # (1) particle encode (2) particle effect
        self.particle_propagator = Propagator(2 * nf_effect, nf_effect)

        # (1) set particle effect
        self.particle_predictor = ParticlePredictor(nf_effect, nf_effect, args.position_dim)
        
        
    def forward(self, attr, state, Rr, Rs, Ra, n_particles, node_r_idx, node_s_idx, pstep,
                instance_idx, phases_dict, verbose=0):
        
        # calculate particle encoding
        if self.use_gpu:
            particle_effect = Variable(torch.zeros((attr.size(0), self.nf_effect)).cuda())
        else:
            particle_effect = Variable(torch.zeros((attr.size(0), self.nf_effect)))
            
        s = 0
        Rrp = Rr[s].t()
        Rsp = Rs[s].t()

        # receiver_attr, sender_attr
        attr_r = attr[node_r_idx[s]]
        attr_s = attr[node_s_idx[s]]
        attr_r_rel = Rrp.mm(attr_r)
        attr_s_rel = Rsp.mm(attr_s)

        # receiver_state, sender_state
        state_r = state[node_r_idx[s]]
        state_s = state[node_s_idx[s]]
        state_r_rel = Rrp.mm(state_r)
        state_s_rel = Rsp.mm(state_s)
        state_diff = state_r_rel - state_s_rel
        
        particle_encode = self.particle_encoder(torch.cat([attr_r, state_r], 1))
        relation_encode = self.relation_encoder(
                torch.cat([attr_r_rel, attr_s_rel, state_r_rel, state_s_rel, Ra[s]], 1))
        
        for i in range(pstep):
            effect_p_r = particle_effect[node_r_idx[s]]
            effect_p_s = particle_effect[node_s_idx[s]]

            receiver_effect = Rrp.mm(effect_p_r)
            sender_effect = Rsp.mm(effect_p_s)
            

            # calculate relation effect
            effect_rel = self.relation_propagator(
                torch.cat([relation_encode, receiver_effect, sender_effect], 1))

            # calculate particle effect by aggregating relation effect
            effect_p_r_agg = Rr[s].mm(effect_rel)

            # calculate particle effect
            effect_p = self.particle_propagator(
                torch.cat([particle_encode, effect_p_r_agg], 1),
                res=effect_p_r)
            particle_effect[node_r_idx[s]] = effect_p
            
        pred = self.particle_predictor(particle_effect[:31])
        return pred

In [31]:
args.pstep = 3
pstep = 3
use_gpu = True
model = DPINet(args, datasets['train'].stat, phases_dict, residual=True, use_gpu=use_gpu)
print("Number of parameters: %d" % count_parameters(model))
# criterion
criterionMSE = nn.MSELoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=(args.beta1, 0.999))
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.8, patience=3, verbose=True)

if use_gpu:
    model = model.cuda()
    criterionMSE = criterionMSE.cuda()

st_epoch = args.resume_epoch if args.resume_epoch > 0 else 0
best_valid_loss = np.inf

model.train(phase=='train')

Number of parameters: 222203


DPINet(
  (particle_encoder): ParticleEncoder(
    (model): Sequential(
      (0): Linear(in_features=7, out_features=100, bias=True)
      (1): ReLU()
      (2): Linear(in_features=100, out_features=150, bias=True)
      (3): ReLU()
    )
  )
  (relation_encoder): RelationEncoder(
    (model): Sequential(
      (0): Linear(in_features=15, out_features=150, bias=True)
      (1): ReLU()
      (2): Linear(in_features=150, out_features=150, bias=True)
      (3): ReLU()
      (4): Linear(in_features=150, out_features=150, bias=True)
      (5): ReLU()
    )
  )
  (relation_propagator): Propagator(
    (linear): Linear(in_features=450, out_features=150, bias=True)
    (relu): ReLU()
  )
  (particle_propagator): Propagator(
    (linear): Linear(in_features=300, out_features=150, bias=True)
    (relu): ReLU()
  )
  (particle_predictor): ParticlePredictor(
    (linear_0): Linear(in_features=150, out_features=150, bias=True)
    (linear_1): Linear(in_features=150, out_features=150, bias=True)
  

In [34]:
phase = 'train'
instance_idx = [0, 31]
args.n_epoch = 10000
psteps = 3

for epoch in range(args.n_epoch):
    model.train(phase=='train')

    losses = 0.
    for i, data in enumerate(dataloaders[phase]):
#         print ('i:',i)

        attr, state, rels, n_particles, n_shapes, label = data
        Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

        Rr, Rs = [], []
        for j in range(len(rels[0])):
            Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
            Rr.append(torch.sparse.FloatTensor(
                Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
            Rs.append(torch.sparse.FloatTensor(
                Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))

        data = [attr, state, Rr, Rs, Ra, label]
        

            # st_time = time.time()
        with torch.set_grad_enabled(phase=='train'):
            if use_gpu:
                for d in range(len(data)):
                    if type(data[d]) == list:
                        for t in range(len(data[d])):
                            data[d][t] = Variable(data[d][t].cuda())
                    else:
                        data[d] = Variable(data[d].cuda())
            else:
                for d in range(len(data)):
                    if type(data[d]) == list:
                        for t in range(len(data[d])):
                            data[d][t] = Variable(data[d][t])
                    else:
                        data[d] = Variable(data[d])

            attr, state, Rr, Rs, Ra, label = data
            
            pstep = 3

            predicted = model(
                attr, state, Rr, Rs, Ra, n_particles,
                node_r_idx, node_s_idx, pstep,
                instance_idx, phases_dict, 0)
            # print('Time forward', time.time() - st_time)

       #     print(predicted.shape)
       #     print(label.shape)

        loss = criterionMSE(predicted, label)
        losses += np.sqrt(loss.item())

        if phase == 'train':
            if i % 5 == 0:
                # update parameters every args.forward_times
          #      print ('update!')
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
          #      print ('done')


        if i % 100000 == 0:
            n_relations = 0
            for j in range(len(Ra)):
                n_relations += Ra[j].size(0)
            print('%s [%d/%d][%d/%d] n_relations: %d, Loss: %.6f, Agg: %.6f' %
                  (phase, epoch, 100, i, len(dataloaders[phase]),
                   n_relations, np.sqrt(loss.item()), losses / (i + 1)))

      #  if phase == 'train' and i > 0 and i % args.ckp_per_iter == 0:
    if epoch % 10 == 0 :
        torch.save(model.state_dict(), '%s/net_epoch_%d.pth' % (args.outf, epoch))

    losses /= len(dataloaders[phase])
    print('%s [%d/%d] Loss: %.4f, Best valid: %.4f' %
          (phase, epoch, args.n_epoch, losses, best_valid_loss))

train [0/100][0/22455] n_relations: 60, Loss: 1.701027, Agg: 1.701027


KeyboardInterrupt: 

## Evaluation

In [14]:
stat_path = stat_path = os.path.join(args.dataf, 'stat.h5')
stat = load_data(data_names[:2], stat_path)

In [22]:
use_gpu = torch.cuda.is_available()
model = DPINet(args, stat, phases_dict, residual=True, use_gpu=use_gpu)

In [23]:
model_file = os.path.join(args.outf, 'net_epoch_%d.pth' % (500))

In [24]:
print("Loading network from %s" % model_file)
model.load_state_dict(torch.load(model_file))
model.eval()

criterionMSE = nn.MSELoss()

if use_gpu:
    model.cuda()

Loading network from dump_SingleHair/files_SingleHair/net_epoch_500.pth


In [8]:
idx = 0


for step in range(args.time_step - 1):
    data_path = os.path.join(args.dataf, 'valid', str(idx), str(step) + '.h5')
    data_nxt_path = os.path.join(args.dataf, 'valid', str(idx), str(step + 1) + '.h5')

    data = load_data(data_names, data_path)
    data_nxt = load_data(data_names, data_nxt_path)
    velocities_nxt = data_nxt[1]

    if step == 0:
        positions, velocities, hairs_idx = data
        n_shapes = 1
        scene_params = 11
        count_nodes = positions.shape[0]
        n_particles = count_nodes - n_shapes
        p_gt = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))
        s_gt = np.zeros((args.time_step - 1, n_shapes, args.shape_state_dim))
        v_nxt_gt = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))

        p_pred = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))

    p_gt[step] = positions[:, -args.position_dim:]
    v_nxt_gt[step] = velocities_nxt[:, -args.position_dim:]
    
    s_gt[step, :, :3] = positions[n_particles:, :3]
    s_gt[step, :, 3:6] = p_gt[max(0, step-1), n_particles:, :3]
    s_gt[step, :, 6:] = np.array( [[0.,0.70710677,0,0.70710677,0,0.70710677, 0, 0.70710677]])
    positions = positions + velocities_nxt * args.dt


In [7]:
step = 0
mass = np.zeros((n_particles, 1))
p = np.concatenate([p_gt[step, :n_particles], mass], 1)
p

array([[0.        , 4.        , 0.        , 0.        ],
       [0.        , 3.88348913, 0.        , 0.        ],
       [0.        , 3.76832294, 0.        , 0.        ],
       [0.        , 3.64844704, 0.        , 0.        ],
       [0.        , 3.53010917, 0.        , 0.        ],
       [0.        , 3.4117341 , 0.        , 0.        ],
       [0.        , 3.29377675, 0.        , 0.        ],
       [0.        , 3.17614532, 0.        , 0.        ],
       [0.        , 3.05889988, 0.        , 0.        ],
       [0.        , 2.94205379, 0.        , 0.        ],
       [0.        , 2.82562876, 0.        , 0.        ],
       [0.        , 2.70964026, 0.        , 0.        ],
       [0.        , 2.59410095, 0.        , 0.        ],
       [0.        , 2.47902036, 0.        , 0.        ],
       [0.        , 2.36440516, 0.        , 0.        ],
       [0.        , 2.25025892, 0.        , 0.        ],
       [0.        , 2.13658237, 0.        , 0.        ],
       [0.        , 2.02337432,

In [9]:
import pyflex
pyflex.init()

cap_size = [0.1,1.5]
N_hairs = 1
env_idx = 12
scene_params = np.array(cap_size)
pyflex.set_scene(env_idx, scene_params, 0)

for step in range(args.time_step - 1):
    pyflex.set_shape_states(s_gt[step])

    mass = np.zeros((n_particles, 1))
    p = np.concatenate([p_gt[step, :n_particles], mass], 1)

    pyflex.set_positions(p)
    pyflex.render()
    
pyflex.clean()

In [49]:
idx = 0
data_path = os.path.join(args.dataf, 'valid', str(idx), '0.h5')
data = load_data(data_names, data_path)
instance_idx = [0, 31]
p_pred = np.zeros((args.time_step - 1, n_particles, args.position_dim))

In [50]:
for step in range(args.time_step - 1):
    p_pred[step] = data[0][:n_particles]
    attr, state, rels, n_particles, n_shapes = prepare_input(data, stat, args, phases_dict, 0)
    Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

    Rr, Rs = [], []
    for j in range(len(rels[0])):
        Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
        Rr.append(torch.sparse.FloatTensor(
            Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
        Rs.append(torch.sparse.FloatTensor(
            Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))

    buf = [attr, state, Rr, Rs, Ra]


        # st_time = time.time()
    with torch.set_grad_enabled(phase=='train'):
        if use_gpu:
            for d in range(len(buf)):
                if type(buf[d]) == list:
                    for t in range(len(buf[d])):
                        buf[d][t] = Variable(buf[d][t].cuda())
                else:
                    buf[d] = Variable(buf[d].cuda())
        else:
            for d in range(len(buf)):
                if type(buf[d]) == list:
                    for t in range(len(buf[d])):
                        buf[d][t] = Variable(buf[d][t])
                else:
                    buf[d] = Variable(buf[d])

        attr, state, Rr, Rs, Ra = buf

        pstep = 3

        predicted = model(
            attr, state, Rr, Rs, Ra, n_particles,
            node_r_idx, node_s_idx, pstep,
            instance_idx, phases_dict, 0)
        
        vels = denormalize([predicted.data.cpu().numpy()], [stat[1]])[0]
        data[0][:n_particles] += vels * args.dt
        data[1][:n_particles] = vels

In [57]:
p_pred[15]

array([[-1.24024999e-04,  4.00099945e+00, -6.39032578e-06],
       [ 5.82799490e-04,  3.89558935e+00, -2.69473330e-05],
       [ 3.54904441e-05,  3.78641129e+00, -5.59073032e-06],
       [ 1.81983385e-04,  3.67839432e+00, -1.61776450e-06],
       [-2.25596479e-04,  3.56450057e+00, -4.03137938e-06],
       [-2.66419403e-04,  3.45700550e+00,  1.20745278e-06],
       [-2.04477197e-04,  3.34188795e+00, -9.99670419e-07],
       [-1.25723836e-05,  3.23407626e+00, -2.90207396e-07],
       [ 2.39915225e-05,  3.12846613e+00, -1.72030730e-06],
       [-7.33436536e-05,  3.01855302e+00, -2.50097582e-06],
       [-1.47372106e-04,  2.90944147e+00, -2.40530676e-06],
       [-1.99186092e-04,  2.80131030e+00, -1.47952471e-06],
       [-2.59405613e-04,  2.69144487e+00, -2.49053573e-06],
       [-3.06985225e-04,  2.58198762e+00, -3.90610649e-06],
       [-3.75870964e-04,  2.47475171e+00, -3.70291468e-06],
       [-3.58385616e-04,  2.36479306e+00, -3.70518319e-06],
       [-4.64003184e-04,  2.24410725e+00

In [ ]:
import pyflex
pyflex.init()

recs = []

for idx in range(3):

    print("Rollout %d / %d" % (idx, 3))

    # ground truth
    for step in range(args.time_step - 1):
        data_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step) + '.h5')
        data_nxt_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step + 1) + '.h5')

        data = load_data(data_names, data_path)
        data_nxt = load_data(data_names, data_nxt_path)
        velocities_nxt = data_nxt[1]

        if step == 0:
            if args.env == 'BoxBath':
                positions, velocities, clusters = data
                n_shapes = 0
                scene_params = np.zeros(1)
            elif args.env == 'FluidFall':
                positions, velocities = data
                n_shapes = 0
                scene_params = np.zeros(1)
            elif args.env == 'RiceGrip':
                positions, velocities, shape_quats, clusters, scene_params = data
                n_shapes = shape_quats.shape[0]
            elif args.env == 'FluidShake':
                positions, velocities, shape_quats, scene_params = data
                n_shapes = shape_quats.shape[0]
            else:
                raise AssertionError("Unsupported env")

            count_nodes = positions.shape[0]
            n_particles = count_nodes - n_shapes
            print("n_particles", n_particles)
            print("n_shapes", n_shapes)

            p_gt = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))
            s_gt = np.zeros((args.time_step - 1, n_shapes, args.shape_state_dim))
            v_nxt_gt = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))

            p_pred = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))

        p_gt[step] = positions[:, -args.position_dim:]
        v_nxt_gt[step] = velocities_nxt[:, -args.position_dim:]

        # print(step, np.sum(np.abs(v_nxt_gt[step, :args.n_particles])))

        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            s_gt[step, :, :3] = positions[n_particles:, :3]
            s_gt[step, :, 3:6] = p_gt[max(0, step-1), n_particles:, :3]
            s_gt[step, :, 6:10] = data[2]
            s_gt[step, :, 10:] = data[2]

        positions = positions + velocities_nxt * args.dt

    # model rollout
    data_path = os.path.join(args.dataf, 'valid', str(infos[idx]), '0.h5')
    data = load_data(data_names, data_path)

    for step in range(args.time_step - 1):
        if step % 10 == 0:
            print("Step %d / %d" % (step, args.time_step - 1))

        p_pred[step] = data[0]

        if args.env == 'RiceGrip' and step == 0:
            data[0] = p_gt[step + 1].copy()
            data[1] = np.concatenate([v_nxt_gt[step]] * (args.n_his + 1), 1)
            continue

        # st_time = time.time()
        attr, state, rels, n_particles, n_shapes, instance_idx = \
                prepare_input(data, stat, args, phases_dict, args.verbose_data)

        Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

        Rr, Rs = [], []
        for j in range(len(rels[0])):
            Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
            Rr.append(torch.sparse.FloatTensor(
                Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
            Rs.append(torch.sparse.FloatTensor(
                Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))

        buf = [attr, state, Rr, Rs, Ra]

        with torch.set_grad_enabled(False):
            if use_gpu:
                for d in range(len(buf)):
                    if type(buf[d]) == list:
                        for t in range(len(buf[d])):
                            buf[d][t] = Variable(buf[d][t].cuda())
                    else:
                        buf[d] = Variable(buf[d].cuda())
            else:
                for d in range(len(buf)):
                    if type(buf[d]) == list:
                        for t in range(len(buf[d])):
                            buf[d][t] = Variable(buf[d][t])
                    else:
                        buf[d] = Variable(buf[d])

            attr, state, Rr, Rs, Ra = buf
            # print('Time prepare input', time.time() - st_time)

            # st_time = time.time()
            vels = model(
                attr, state, Rr, Rs, Ra, n_particles,
                node_r_idx, node_s_idx, pstep,
                instance_idx, phases_dict, args.verbose_model)
            # print('Time forward', time.time() - st_time)

            # print(vels)

            if args.debug:
                data_nxt_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step + 1) + '.h5')
                data_nxt = normalize(load_data(data_names, data_nxt_path), stat)
                label = Variable(torch.FloatTensor(data_nxt[1][:n_particles]).cuda())
                # print(label)
                loss = np.sqrt(criterionMSE(vels, label).item())
                print(loss)

        vels = denormalize([vels.data.cpu().numpy()], [stat[1]])[0]

        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            vels = np.concatenate([vels, v_nxt_gt[step, n_particles:]], 0)
        data[0] = data[0] + vels * args.dt

        if args.env == 'RiceGrip':
            # shifting the history
            # positions, restPositions
            data[1][:, args.position_dim:] = data[1][:, :-args.position_dim]
        data[1][:, :args.position_dim] = vels

        if args.debug:
            data[0] = p_gt[step + 1].copy()
            data[1][:, :args.position_dim] = v_nxt_gt[step]

    ##### render for the ground truth
    pyflex.set_scene(env_idx, scene_params, 0)

    if args.env == 'RiceGrip':
        halfEdge = np.array([0.15, 0.8, 0.15])
        center = np.array([0., 0., 0.])
        quat = np.array([1., 0., 0., 0.])
        pyflex.add_box(halfEdge, center, quat)
        pyflex.add_box(halfEdge, center, quat)
    elif args.env == 'FluidShake':
        x, y, z, dim_x, dim_y, dim_z, box_dis_x, box_dis_z = scene_params
        boxes = calc_box_init_FluidShake(box_dis_x, box_dis_z, height, border)

        x_box = x + (dim_x-1)/2.*0.055

        for box_idx in range(len(boxes) - 1):
            halfEdge = boxes[box_idx][0]
            center = boxes[box_idx][1]
            quat = boxes[box_idx][2]
            pyflex.add_box(halfEdge, center, quat)


    for step in range(args.time_step - 1):
        if args.env == 'RiceGrip':
            pyflex.set_shape_states(s_gt[step])
        elif args.env == 'FluidShake':
            pyflex.set_shape_states(s_gt[step, :-1])

        mass = np.zeros((n_particles, 1))
        if args.env == 'RiceGrip':
            p = np.concatenate([p_gt[step, :n_particles, -3:], mass], 1)
        else:
            p = np.concatenate([p_gt[step, :n_particles], mass], 1)

        pyflex.set_positions(p)
        pyflex.render(capture=0)

    ##### render for the predictions
    pyflex.set_scene(env_idx, scene_params, 0)

    if args.env == 'RiceGrip':
        pyflex.add_box(halfEdge, center, quat)
        pyflex.add_box(halfEdge, center, quat)
    elif args.env == 'FluidShake':
        for box_idx in range(len(boxes) - 1):
            halfEdge = boxes[box_idx][0]
            center = boxes[box_idx][1]
            quat = boxes[box_idx][2]
            pyflex.add_box(halfEdge, center, quat)

    for step in range(args.time_step - 1):
        if args.env == 'RiceGrip':
            pyflex.set_shape_states(s_gt[step])
        elif args.env == 'FluidShake':
            pyflex.set_shape_states(s_gt[step, :-1])

        mass = np.zeros((n_particles, 1))
        if args.env == 'RiceGrip':
            p = np.concatenate([p_pred[step, :n_particles, -3:], mass], 1)
        else:
            p = np.concatenate([p_pred[step, :n_particles], mass], 1)

        pyflex.set_positions(p)
        pyflex.render(capture=0)

pyflex.clean()